# 7.2 심층 신경망_deep neural network, DNN


- 딥러닝(deep learning): 딥러닝은 인공신경망과 거의 동의어로 사용되는 경우가 많으며 혹은 심층 신경망을 딥러닝이라고 부름. 심층 신경망은 여러 개의 층을 가진 인공신경망임.


- 심층 신경망(deep neural network, DNN): 2개 이상의 층을 포함한 신경망으로 종종
다층 인공신경망, 심층 신경망, 딥러닝을 같은 의미로 사용함.

- 렐루 함수(ReLU Function): 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼
그냥 입력을 통과시키기고 음수일 경우에는 0으로 만드는 함수

- 옵티마이저(optimizer): 신경망의 가중치와 절편을 학습하기 위한 알고리즘 또는 방법. 케라스에는 다양한 경사 하강법 알고리즘이 구현되어 있으며 대표적으로 SGD, 네스테로프 모멘텀, RMSprop, Adam 등이 있음

In [1]:
# 라이브러리 호출 및 데이터 준비
from tensorflow import keras        # 텐서플로를 사용해 데이터 불러오기
from sklearn.model_selection import train_test_split    # 교차 검증으로 성능을 확인

(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
    # 데이터셋 불러오기, train 과 test set 으로 나누기

train_scaled = train_input /255.0
    # 픽셀 값을 0~255 범위에서 0~1 사이로 변환

train_scaled = train_scaled.reshape(-1, 28*28)
    # 28x28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼치기
    
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)
    # train을 train과 validation으로 나누기

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# 모델에 층을 추가하는 방법 1

# 인공 신경망 모델에 층을 2개 추가
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
    # dense1는 은닉층으로, 100개의 뉴런을 가짐, 활성화 함수를 sigmoid로 지정, input 매개변수에서 입력 크기를 784로 지정함.
    # 은닉층의 뉴런 개수를 정하는데 특별한 기준은 없으나 몇 개의 뉴런을 두어야 할 지 결정하는 것은 경험이 필요함
    # 은닉층의 뉴런 개수는 적어도 출력층의 뉴런보다 많아야 함.

dense2 = keras.layers.Dense(10, activation='softmax')
    # 출력층, 10개의 클래스를 분류하므로 10개의 뉴런을 두었고 활성화 함수는 softmax로 지정.


# 심층 신경망 만들기: 앞서 만든 dense1과 dense2를 Sequential 클래스에 추가함.
model1 = keras.Sequential([dense1, dense2])

model1.summary() # 케라스 모델의 summary()메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있음.
    # 케라스 모델의 fit 메서드에 훈련 데이터를 주입하면 이 데이터를 잘게 나누어 여러번 걸쳐 경사 하강법 단계를 수행함.
    # 미니배치 경사 하강법으로, 기본 미니배치 크기는 32개

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델에 층을 추가하는 방법 2
    # Dense1, 2 두 객체를 따로 저장하여 쓸 일이 없기 때문에
    # Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많음
    
model2 = keras.Sequential([
    keras.layers.Dense(100,activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')], 
    name='MNIST model')
    # model1과 달리 model2는 Sequential 클래스의 name 매개변수로 모델의 이름과 dense층의 이름을 지정.

model2.summary() 
    # 이 방법은 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어짐.
    # 조건에 따라 층을 추가할 수도 없음. 


Model: "MNIST model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델에 층을 추가하는 방법 3: Sequential 클래스에서 층을 추가할 때 가장 널리 사용하는 방법- add() 메서드
model3 = keras.Sequential()
model3.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model3.add(keras.layers.Dense(10, activation='softmax'))

model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 훈련하기: compile() 메서드
model3.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model3.fit(train_scaled, train_target, epochs=5)
model3.evaluate(val_scaled, val_target)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5709 - accuracy: 0.8052
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4086 - accuracy: 0.8524
Epoch 3/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3726 - accuracy: 0.8665
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3507 - accuracy: 0.8722
Epoch 5/5
375/375 [==============================] - 1s 1ms/step - loss: 0.3558 - accuracy: 0.8689


[0.3558274209499359, 0.8689166903495789]

In [ ]:
# 모델 훈련하기_ 케라스에서는 Flatten class
    # 패션 MNIST 데이터는 28x28 크기이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape() 메서드를 사용해 1차원으로 펼침
    # 케라스에서는 Flatten class는 배치자원을 제외하고 나머지 입력차원을 모두 일렬로 펼침.
    #   인공 신경망의 성능을 위해 기여하는 바가 없고, 입력층과 은닉층 사이에 추가 하기 떄문에 Flatten 층이라고 함.
    #   Flatten층은 다음 코드처럼 입력층 바로 뒤에 추가함.
model4 = keras.Sequential()
model4.add(keras.layers.Flatten(input_shape=(28,28)))
    # Flatten 클래스에 포함된 모델 파라미터는 0개. 
    # 케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 장점.
model4.add(keras.layers.Dense(100, activation='relu'))
model4.add(keras.layers.Dense(10, activation='softmax'))

model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


## ** 렐루 함수 **

- 패션 MNIST 데이터는 28x28 크기이기 때문에 인공 신경망에 주입하기 위해 넘파이 배열의 reshape() 메서드를 사용해 1차원으로 펼침
- 케라스에서는 Flatten class는 배치자원을 제외하고 나머지 입력차원을 모두 일렬로 펼침.
- 인공 신경망의 성능을 위해 기여하는 바가 없고, 입력층과 은닉층 사이에 추가 하기 떄문에 Flatten 층이라고 함.
- Flatten층은 다음 코드처럼 입력층 바로 뒤에 추가함.

In [ ]:
model5 = keras.Sequential()
model5.add(keras.layers.Flatten(input_shape=(28,28)))
model5.add(keras.layers.Dense(100, activation='relu'))
model5.add(keras.layers.Dense(10, activation='softmax'))

model5.summary()
    # Flatten 클래스에 포함된 모델 파라미터는 0개.
    # 케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 장점.
    # Flatten층에서 입력값 784개의 입력이 그대로 첫 번째 은닉층에 전달되는 것을 확인
    # 케라스 API는 입력 데이터에 대한 전처리 과정을 가능한 모델에 포함시킴.


Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_17 (Dense)             (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 훈련: 렐루 함수 적용시, reshpae() 적용하지 않음
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()    # 데이터셋 불러오기, train 과 test set 으로 나누기

train_scaled = train_input /255.0               # 픽셀 값을 0~255 범위에서 0~1 사이로 변환
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)     # train을 train과 validation으로 나누기

model5.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model5.fit(train_scaled, train_target, epochs=5)

model5.evaluate(val_scaled, val_target)
  # 렐루 함수 적용 전보다 적용 후에 정확도가 향상됨. 

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3133 - accuracy: 0.8905
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3027 - accuracy: 0.8945
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2941 - accuracy: 0.8976
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2886 - accuracy: 0.8986
Epoch 5/5
375/375 [==============================] - 1s 1ms/step - loss: 0.3895 - accuracy: 0.8810


[0.3895147740840912, 0.8809999823570251]

In [ ]:
# Optimizer를 *Adam* 클래스 설정(기본 RMSprop 사용)

# 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련
model6 = keras.Sequential()
model6.add(keras.layers.Flatten(input_shape=(28,28)))
model6.add(keras.layers.Dense(100, activation='relu'))
model6.add(keras.layers.Dense(10, activation='softmax'))

# 모델 훈련: 렐루 함수 적용시, reshpae() 적용하지 않음
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()    # 데이터셋 불러오기, train 과 test set 으로 나누기

train_scaled = train_input /255.0               # 픽셀 값을 0~255 범위에서 0~1 사이로 변환
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)     # train을 train과 validation으로 나누기

model6.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

model6.fit(train_scaled, train_target, epochs=5)

model6.evaluate(val_scaled, val_target)
  # 환경마다 조금씩 차이가 있지만 여기서는 기본 RMSprop보다 조금 나은 성능을 나타냄

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5169 - accuracy: 0.8209
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3907 - accuracy: 0.8605
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3476 - accuracy: 0.8733
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3224 - accuracy: 0.8824
Epoch 5/5
375/375 [==============================] - 1s 1ms/step - loss: 0.3358 - accuracy: 0.8788


[0.33583638072013855, 0.8787500262260437]